In [5]:
import cv2
import numpy as np
import mediapipe as mp

counter = 0
direction = 0

cap = cv2.VideoCapture(0)  # Using webcam for live feed
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def calculate_angle(a, b, c):
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle 

while True:
    ret, img = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        lmlist = results.pose_landmarks.landmark
        height, width, _ = img.shape
        
        # Extracting landmark coordinates
        landmarks = {}
        for i, landmark in enumerate(lmlist):
            landmarks[i] = (int(landmark.x * width), int(landmark.y * height))
        
        # Calculate angles
        left_shoulder = landmarks[11]
        left_elbow = landmarks[13]
        left_wrist = landmarks[15]
        right_shoulder = landmarks[12]
        right_elbow = landmarks[14]
        right_wrist = landmarks[16]

        lefthandangle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        righthandangle = calculate_angle(right_shoulder, right_elbow, right_wrist)

        # Update counter and direction
        if lefthandangle > 70 and righthandangle > 70:
            if direction == 0:
                counter += 0.5
                direction = 1
        if lefthandangle <= 70 and righthandangle <= 70:
            if direction == 1:
                counter += 0.5
                direction = 0

        # Visualize joints
        for landmark in landmarks.values():
            cv2.circle(img, landmark, 5, (0, 255, 0), -1)

        # Draw lines connecting the points
        connections = [(11, 13), (13, 15), (12, 14), (14, 16)]
        for connection in connections:
            cv2.line(img, landmarks[connection[0]], landmarks[connection[1]], (0, 0, 255), 3)

        # Visualize counter
        cv2.rectangle(img, (0, 0), (120, 120), (255, 0, 0), -1)
        cv2.putText(img, str(int(counter)), (20, 70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.6, (0, 0, 255), 7)

        # Display the image
        cv2.imshow('frame', img)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()